In [14]:
import json
from grab import Grab
import os
import sqlite3
import datetime

# Создаем список всех городов
with open('C:\\Users\\admin\\Downloads\\city.list.json\\city.list.json', 'r', encoding = 'UTF-8') as f:
    read_data = json.load(f)
city_list = []
for i in read_data:
    city_list.append(i['name'])
city = set(filter(len, city_list))

# Проверяем: имеется ли уже база данных или нет. Если нет - создаем.
if os.path.isfile('weathetoday10.db'):
    db_filename = 'weathetoday10.db'
else:
    db_filename = 'weathetoday10.db'
    conn = sqlite3.connect(db_filename)
    conn.close()
    with sqlite3.connect(db_filename) as conn:
        conn.execute("""
            create table project (
                id_города           INTEGER PRIMARY KEY,
                Город               VARCHAR(255),
                Дата                DATE,
                Температура         INTEGER,
                id_погоды           INTEGER  
            );
            """)

# Запрос города. Если он есть в базе, запрашиваем на openweathermap данные и добавляем в локальную базу данных
answer = input('Введите название города, в котором Вы хотите узнать погоду (формат ввода: Hurzuf): ')
if answer in city:
    print('Подождите. Запрашиваю данные о погоде в городе: ', answer)
    with open('C:\\Users\\admin\\Downloads\\city.list.json\\city.list.json', 'r', encoding = 'UTF-8') as n:
        read_data = json.load(n)
    for i in read_data:
        if i['name'] == answer:
            g = Grab()
            g.setup(url = 'http://api.openweathermap.org/data/2.5/weather?id='+str(i['id'])+'&appid=ecbb6bb1b315dd3a3af9468dc6060163', log_file ='pogoda1.json')
            g.request()
            with open('C:\\Users\\admin\\pogoda1.json', 'r', encoding = 'UTF-8') as n:
                read_data1 = json.load(n)
            t = Grab()
            t.setup(url = 'http://api.openweathermap.org/data/2.5/weather?id='+str(i['id'])+'&units=metric&appid=ecbb6bb1b315dd3a3af9468dc6060163', log_file ='tempra1.json')
            t.request()
            with open('C:\\Users\\admin\\tempra1.json', 'r', encoding = 'UTF-8') as k:
                read_data2 = json.load(k)
            with sqlite3.connect(db_filename) as conn:
                conn.execute("""
                    insert or replace into project (id_города, Город, Дата, Температура,id_погоды) VALUES (?,?,?,?,?)""", (
                        read_data1['id'], 
                        read_data1['name'], 
                        datetime.date.today(),
                        read_data2['main']['temp'],
                        read_data2['weather'][0]['id']
                      )
                )
            with sqlite3.connect(db_filename) as conn:
                conn.row_factory = sqlite3.Row
                cur = conn.cursor()
                cur.execute("select * from project")
                for row in cur.fetchall():
                    #print(row)
                    id_города, Город, Дата, Температура,id_погоды = row
                    print('id_города:',id_города, 'Город:',Город, 'Дата:',Дата, 'Температура:',Температура,'id_погоды:',id_погоды)
else:
    print('В базе нет города с таким названием. Проверьте правильность транскрипции названия города на латынь и попробуйте еще раз.')

Введите название города, в котором Вы хотите узнать погоду (формат ввода: Hurzuf): Krasnodar
Подождите. Запрашиваю данные о погоде в городе:  Krasnodar
id_города: 538560 Город: Kursk Дата: 2018-08-19 Температура: 25 id_погоды: 800
id_города: 542420 Город: Krasnodar Дата: 2018-08-19 Температура: 28 id_погоды: 800
id_города: 703448 Город: Kiev Дата: 2018-08-19 Температура: 25.06 id_погоды: 800
id_города: 2013348 Город: Vladivostok Дата: 2018-08-19 Температура: 18 id_погоды: 800
